# Convert raw portrait capture to images

## Video requirement
The rignerf expects 3 types of videos for each subject under each scene. Each video should be around one minute long. The videos should be in the following format:
- In this first video, the subject should maintain still and the camera should move around the subject. 
- In the second video, the subject should enact a wide range of expressions and speech while trying to keep their head still as the camera is panned around them
- In the third video, the subject should make a variety of facial expressions and head poses, while the camera should be fixed.


## Declare constants

In [1]:
capture_name = 'tianyu_home'
downscale_size = 512
target_frames_per_video_1 = 700
target_frames_per_video_2 = 1500
target_frames_per_video_3 = 1500
LAPLACIAN_THRESHOLD = 20
flip = True

In [2]:
import os
root_dir = os.path.join('./',capture_name)
video_path = os.path.join(root_dir, 'videos')
video_path_1 = os.path.join(video_path, 'capture1.MOV')
video_path_2 = os.path.join(video_path, 'capture2.MOV')
video_path_3 = os.path.join(video_path, 'capture3.MOV')
rgb_dir = os.path.join(root_dir, 'rgb')
raw_rgb_dir = os.path.join(rgb_dir, '1x')
rgb_ds_dir = os.path.join(rgb_dir, 'ds'.format(downscale_size))

## Flatten into images and remove blurry images

In [3]:
import cv2
import os
def flatten_video(video_path, rgb_dir, raw_rgb_dir, rgb_ds_dir, target_frames_per_video, part):
    # Create directories
    if not os.path.exists(rgb_dir):
        os.makedirs(rgb_dir)
    if not os.path.exists(raw_rgb_dir):
        os.makedirs(raw_rgb_dir)
    if not os.path.exists(rgb_ds_dir):
        os.makedirs(rgb_ds_dir)
    print('Flattening {} to {}'.format(video_path, rgb_dir))
    cap = cv2.VideoCapture(str(video_path))
    input_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if frame_count < target_frames_per_video:
        raise RuntimeError('The video is too short and has fewer frames than the target.')
    # hyper-parameters

    success,image = cap.read()
    count = 0
    skip = round(frame_count / target_frames_per_video)
    skip_count = 0
    print('Input FPS: {}'.format(input_fps))
    print('Frame count: {}'.format(frame_count))
    print('Skip: {}'.format(skip))
    blurry_count = 0
    while success:
        # get frame from video
        success,image = cap.read()
        if not success:
            break
        if skip_count < skip:
            skip_count += 1
            continue
        skip_count = 0
        if cv2.Laplacian(image, cv2.CV_64F).var() < LAPLACIAN_THRESHOLD: # check if image is blurry, if so, skip this frame
            blurry_count += 1
            continue
        height, width = image.shape[:2] 
        new_height, new_width = min(height, width), min(height, width)
        if flip:
            image = cv2.flip(image, -1)
        # crop image to square
        image = image[(height-new_height)//2:(height-new_height)//2+new_height, (width-new_width)//2:(width-new_width)//2+new_width]
        image_resize = cv2.resize(image, (downscale_size, downscale_size), interpolation = cv2.INTER_AREA)
        if part == 1:
            cv2.imwrite(os.path.join(raw_rgb_dir,f"p1_{count:06d}.png"),image)     # save frame as png file     
            cv2.imwrite(os.path.join(rgb_ds_dir,f"p1_{count:06d}.png"),image_resize)   # save frame as png file      
        elif part == 2:
            cv2.imwrite(os.path.join(raw_rgb_dir,f"p2_{count:06d}.png"),image)     # save frame as png file     
            cv2.imwrite(os.path.join(rgb_ds_dir,f"p2_{count:06d}.png"),image_resize)   # save frame as png file 
        elif part == 3:
            cv2.imwrite(os.path.join(raw_rgb_dir,f"p3_{count:06d}.png"),image)
            cv2.imwrite(os.path.join(rgb_ds_dir,f"p3_{count:06d}.png"),image_resize)
        count += 1
    print('Failed to read a frame, current count =', count)
    print('Blurry count: {}'.format(blurry_count))


In [4]:
# process three videos
flatten_video(video_path_1, rgb_dir, raw_rgb_dir, rgb_ds_dir, target_frames_per_video_1, 1)
flatten_video(video_path_2, rgb_dir, raw_rgb_dir, rgb_ds_dir, target_frames_per_video_2, 2)
flatten_video(video_path_3, rgb_dir, raw_rgb_dir, rgb_ds_dir, target_frames_per_video_3, 3)

Flattening ./tianyu_home/videos/capture1.MOV to ./tianyu_home/rgb
Input FPS: 29.97351816131036
Frame count: 1528
Skip: 2
Failed to read a frame, current count = 508
Blurry count: 1
Flattening ./tianyu_home/videos/capture2.MOV to ./tianyu_home/rgb
Input FPS: 29.758372191606615
Frame count: 1521
Skip: 1
Failed to read a frame, current count = 751
Blurry count: 4
Flattening ./tianyu_home/videos/capture3.MOV to ./tianyu_home/rgb
Input FPS: 29.973409493795547
Frame count: 2029
Skip: 1
Failed to read a frame, current count = 1014
Blurry count: 0


In [16]:
set(os.listdir(rgb_ds_dir)).difference(set(os.listdir(raw_rgb_dir)))

set()

In [17]:
set(os.listdir(raw_rgb_dir)).difference(set(os.listdir(rgb_ds_dir)))

set()